In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout
import matplotlib.pyplot as plt
from util import plot_history
import feature_builder

Cargo datasets

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [19]:
kernel_size=5

Armo features con el feature_builder

In [21]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1])

In [22]:
X_train = prepare_for_cnn(train_df)
y_train = train_df['target'].values

Percentage of words covered in the embeddings = 0.6336399642263958


Creo el modelo y corro la CNN

In [23]:
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=kernel_size, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=128, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(1e-4), metrics=['accuracy'])
    return model

In [24]:
model = create_model()
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=3)
history = model.fit(X_train, y_train, epochs=30, shuffle=True, validation_split=0.2, callbacks=[es])
plot_history(history)

ValueError: Negative dimension size caused by subtracting 5 from 1 for '{{node conv1d_4/conv1d}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv1d_4/conv1d/ExpandDims, conv1d_4/conv1d/ExpandDims_1)' with input shapes: [?,1,1,320], [1,5,320,128].

Calculemos el test para kaggle

In [ ]:
ids = test_df['id']

final_test = prepare_for_cnn(test_df)
prediction = model.predict_classes(final_test)

final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

In [ ]:
final_df['target'].value_counts()

In [ ]:
final_df.count()

In [ ]:
final_df.to_csv('cnn.csv')